# Homework 3 - Places of the world


## 1. Data collection

For this homework, there is no provided dataset. Instead, you have to build your own. Your search engine will run on text documents. So, here
we detail the procedure to follow for the data collection. 
 

### IMPORT

In [14]:
import requests
from bs4 import BeautifulSoup

### 1.1. Get the list of places

We start with the list of places to include in your corpus of documents. In particular, we focus on the [Most popular places](https://www.atlasobscura.com/places?sort=likes_count). Next, we want you to **collect the URL** associated with each site in the list from this list.
The list is long and split into many pages. Therefore, we ask you to retrieve only the URLs of the places listed in **the first 400 pages** (each page has 18 places, so that you will end up with 7200 unique place URLs).

The output of this step is a `.txt` file whose single line corresponds to the place's URL.


Built the urls i need to get every url for every places.

In [15]:
url  = 'https://www.atlasobscura.com/places?sort=likes_count'
main_domain  ='https://www.atlasobscura.com'
get_page_query = '/places?page='
query_end = '&sort=likes_count'

Main code to get every url of evry places and the save them locally in places_url.txt

In [ ]:
places_url  = []
output = open("places_url.txt",'w')
cnt  =  0
for i in range(1,401):
    try:
        req  = requests.get(main_domain+get_page_query+str(i)+query_end)
        soup  = BeautifulSoup(req.text)
        places_url= [main_domain+ x.get('href') for x in soup.find_all('a',{'class' : 'content-card-place' })]
        output.write('\n'.join(places_url))
        cnt+=18
    except Exception as e:
        print("Error Occured: "+e)
output.close()

Check if all the 7200 urls have been written in places_url.txt

In [17]:
print(cnt)

7200


### 1.2. Crawl places

Once you get all the URLs in the first 400 pages of the list, you:

1. Download the HTML corresponding to each of the collected URLs.
2. After you collect a single page, immediately save its `HTML` in a file. In this way, if your program stops for any reason, you will not lose the data collected up to the stopping point.
3. Organize the entire set of downloaded `HTML` pages into folders. Each folder will contain the `HTML` of the places on page 1, page 2, ... of the list of locations.

__Tip__: Due to a large number of pages you should download, you can use some methods that can help you shorten the time it takes. If you employed a particular process or approach, kindly describe it.

#### IMPORT

In [1]:
import linecache
import requests
import os

### get_html function

In this function there is the code to download the page html of a place on www.atlasobscura.com:

*Parameters*

    __url__ : this url is the location of the html page that we want to download, it is read from a file txt containing 7200 url of places on www.atlasobscura.com.
    
    __path__: this path is the location where we want to save the .html file locally.

    __number__: this number goes from 0 to 7199, each number corresponds to a different place.
    
    __page__: this number goes from 1 to 400, each number corresponds to a page from this link https://www.atlasobscura.com/places?page=<page>sort=likes_count.

*Execution*
    
    This function makes a GET request to the given <url>, tries to create a new html file called place<number>.html in the <path> given as input.
    When the GET request has a status code equals to 200, the request.text attribute is written in the place<number>.html. 
    After the file is written, it gives a feedback if the file as already been saved locally( in this case you will see on the standard output "Done!") or if it has just been saved locally(in this case you will see on the standard output "Downloaded place <number>, Page <page>).

In [2]:
def get_html(url,path,number,page):
    try:    
        with open(path+"/place"+str(number)+".html","w", encoding="utf-8") as file_html:
            req =  requests.get(url)
            if req.status_code == 200:
                file_html.write(req.text)
                print("Downloaded place "+str(number)+", Page "+str(page))
    except Exception as e:
        print("Error occured! "+ str(e))

This block of code creates the need 400 directory correponding to each page from this link  https://www.atlasobscura.com/places?page=<page_index>sort=likes_count.

In [3]:
for page_index  in range(1,401):
    dir_path  = "HTML_FILES/page"+str(page_index)
    if not os.path.exists(dir_path):
            os.makedirs(dir_path)

Main code the calls get_html for every url saved in the places_url.txt.
I used linecache module to acces directly to a specific line in the file.txt.

In [12]:
for i in range(0,7200):
    page  = (i//18)+1
    dir_path  = "HTML_FILES/page"+str(page)
    url = str(linecache.getline('places_url.txt',i+1).replace('\n',""))
    if not os.path.exists(dir_path+"/place"+str(i)+".html") or os.path.getsize(dir_path+"/place"+str(i)+".html") < 2000:
        get_html(url,dir_path,i,page)

### 1.3 Parse downloaded pages

At this point, you should have all the HTML documents about the places of interest, and you can start to extract the places' information. The list of the information we desire for each place and their format is as follows:

1. Place Name (to save as `placeName`): String.
2. Place Tags (to save as `placeTags`): List of Strings.
3. \# of people who have been there (to save as `numPeopleVisited`): Integer.
4. \# of people who want to visit the place(to save as `numPeopleWant`): Integer.
5. Description (to save as `placeDesc`): String. Everything from under the first image up to "know before you go" (orange frame on the example image).
6. Short Description (to save as `placeShortDesc`): String. Everything from the title and location up to the image (blue frame on the example image).
7. Nearby Places (to save as `placeNearby`): Extract the names of all nearby places, but only keep unique values: List of Strings.
8. Address of the place(to save as `placeAddress`): String.
9. Altitude and Longitude of the place's location(to save as `placeAlt` and `placeLong`): Integers
10. The username of the post editors (to save as `placeEditors`): List of Strings.
11. Post publishing date (to save as `placePubDate`): datetime.
12. The names of the lists that the place was included in (to save as `placeRelatedLists`): List of Strings.
13. The names of the related places (to save as `placeRelatedPlaces`): List of Strings.
14. The URL of the page of the place (to save as `placeURL`):String
<p align="center">
<img src="img/last_version_place.png" width = 1000>
</p>


For each place, you create a `place_i.tsv` file of this structure:

```
placeName \t placeTags \t  ... \t placeURL
```

If an information is missing, you just leave it as an empty string.


#### IMPORT FOR WEB-SCRAPING

All the functions we used for web_scraping are written in the web_scraping_functions.py file 

In [5]:
from web_scraping_functions import *
import linecache

#### CREATE TSV FILES FOR EACH PLACES IN 'https://www.atlasobscura.com/places'

First we create a directory for each page in 'https://www.atlasobscura.com/places', example: for *page 1* the directory will be named *page1*.

All the directories will be inside a main directory called **TSV_FILES**.

The tsv file for the *i-th place* is named as so: *"place_i.tsv"*. Each tsv file is saved in the directory corresponding to the page in the site where you can find the *i-th place*. 

##### EXPLANATION OF MAKE_PLACE_TSV FUNCTION: 
    
After creating all the directories, it start a **for** loop to create for each place a tsv file, so **i** it goes from 0 to 7200.


In each iteration of the **for** loop, we create the parameters for the make_place_tsv functions:

**i** is the number of the place
        
 **page** is an integer representing the number of the page in which is published the current place, we need this number to build the html path.
 
 **html_path** is the path in the homework folder where all the html file are stored, ordered by page.

 **tsv_path** is the path in the homework folder where we want to create the tsv file for the **i**-th place.



*def make_place_tsv(place_number,tsv_path,html_path,url):*  definition of make_place_tsv, written in web_scraping_functions.py

**OUTPUT:** the place_**i**.tsv is created in the **TSV_FILES**/page**page** path. Structure of the tsv file created: placeName \t placeTags \t  ... \t placeURL

***SOME STATS:***   It took about 18 minutes to create 7200 tsv files.

The function is written below, all the other functions used inside are written in web_scraping_functions.py. 

In [13]:
def make_place_tsv(place_number,tsv_path,html_path,url):
    if not os.path.exists(tsv_path+"/place_"+str(place_number)+'.tsv'): 
        print("Place "+str(place_number)+"Page "+str((place_number//18)+1))
        try:
            with open(html_path,"r",encoding='utf-8') as html:
                soup = BeautifulSoup(html,'lxml')
                row = ""
                row += str(get_placeName(soup))+' \t'
                row += str(get_placeTags(soup))+' \t'
                row += str(get_placePeopleVisited(soup))+' \t'
                row += str(get_placePeopleWant(soup))+' \t'
                row += ' '+str(get_placeDesc(soup))+' \t'
                row += ' '+str(get_placeShortDesc(soup))+' \t'
                row+=' '+str(get_placeNearby(soup))+' \t'
                row+=' '+str(get_placeAddress(soup))+' \t'
                placeAlt,placeLong = get_placeCoordinates(soup)
                row +=' '+str(placeAlt)+' \t'
                row +=' '+str(placeLong)+' \t'
                row +=' '+str(get_placeEditors(soup))+' \t'
                row +=' '+str(get_placePubDate(soup))+' \t'
                row +=' '+str(get_placeRelatedLists(soup))+' \t'
                row+=' '+str(get_placeRelatedPlaces(soup))+' \t'
                row += ' '+url+' \t'
                #print(row)
                try:
                        with open(tsv_path+"/place_"+str(place_number)+'.tsv','w',encoding='utf-8') as f:
                            f.write(row)
                            print("csv created!")
                except  Exception as e:
                    print("File not Created: "+e)
        except Exception as e:
            print("Error occurred: "+str(e))

In [15]:
create_tsv_directories()
for i in range(0,7200):
    page  = (i//18)+1
    html_path = "HTML_FILES/page"+str(page)+"\place"+str(i)+".html"
    tsv_path  = "TSV_FILES/page"+str(page)
    url  = str(linecache.getline('places_url.txt',i+1)).replace('\n',"")
    make_place_tsv(i+1,tsv_path,html_path,url)

#### MERGE ALL THE TSV FILES IN ONE

In this csv file each row represents a place and has this structure: 
     
     
     placeName \t placeTags \t placePeopleVisited \t placePeopleWant \t placeDesc \t placeShortDesc \t placeNearby \t placeAddress \t placeAlt \t placeLong \t placeEditors \t placePubDate \t placeRelatedLists \t placeRelatedPlaces \t placeURL
    

In [16]:
def merge_tsv():
    with open('merged.tsv','w',encoding='utf-8') as merged:
        merged.write('placeName \t placeTags \t placePeopleVisited \t placePeopleWant \t placeDesc \t placeShortDesc \t placeNearby \t placeAddress \t placeAlt \t placeLong \t placeEditors \t placePubDate \t placeRelatedLists \t placeRelatedPlaces \t placeURL\n')
        for i in range(0,7200):
            page  = (i//18)+1
            tsv_file  = "TSV_FILES/page"+str(page)+'/place_'+str(i+1)+'.tsv'
            with open(tsv_file,'r',encoding='utf-8') as tsv : 
                merged.write(tsv.readline()+'\n')

In [17]:
merge_tsv()

## 7. Theoretical question
An imaginary university is interested in accepting some of the applicants for positions to study the Master of Data Science there. Unfortunately, only a few spots are available, so the university requires students to take some exams. Students are then admitted based on how well they perform on these exams. For students to determine whether they have been successfully accepted to the university, the university wants to create a ranking list that includes every student's first name, last name, and total average on its course webpage. Students should be ranked in the list based on their average points in descending order. For example, if two students have the same average punctuation, they should be sorted in ascending order using their first and last names. University will give you the students' information in __'ApplicantsInfo.txt'__ ([click here to download](https://adm2022.s3.amazonaws.com/ApplicantsInfo.txt)), and you should provide them with the ranking list in another *.txt* file and name it as __'RankingList.txt'__ . Kindly help this university in preparing this ranking list.

**Input:** 
__'ApplicantsInfo.txt'__ will have the following format: 
- In the first line, you will be given *n* as the number of applicants and *m* as the number of exams that students have taken (all of them have taken the same exams), where: 
$$0 \lt n \le 5 * 10^4$$
$$1 \le m \le 10^3$$
- In each following *n* lines, you will find the information related to one of the students. Their first name, last name and *m* integers as the grades they received in *m* courses. 
 
**Output:**
The output file should consist of __n__ lines, with each line representing one of the students and including the student's __first name, last name, and total average point__(setting the precision to 2 decimal points). As you know, they must be sorted in the order specified in the problem description. 


1. Try solving the problem mentioned above using three different sorting algorithms (do not use any MapReduce algorithm). (__Note:__ Built-in Python functions (like .mean, .sort, etc.) are not allowed to be used. You must implement the algorithms from scratch).
2. What is the time complexity of each algorithm you have used?
3. Evaluate the time taken for each of your implementations to answer the query stored in the __ApplicantsInfo.txt__ file and visualize them.
4. What is the most optimal algorithm, in your opinion, and why?
5. Implement a sorting algorithm using MapReduce and compare it against the three algorithms previously implemented using the __ApplicantsInfo.txt__ file.

### 7.1

**Sorting Algorithms**

To sort all the rows in the ___ApplicantsInfo.txt__, we implemented 3 sorting algorithm:

1. __SelectionSort__
2. __BubbleSort__
3. __MergeSort__

In [6]:
def sel_sort(values,N):
	for i in range(N):
		max_i = i
		for  j in range(i+1,N):
			if( values[max_i][2] < values[j][2]):
				max_i = j
			elif (values[max_i][2] == values[j][2]):
				if((values[max_i][0]+values[max_i][1]).lower() > (values[j][0] +values[j][1]).lower()):
					max_i = j
		values[i],values[max_i] = values[max_i],values[i]

def swap(i,j,values):
	temp =  values[i]
	values[i] = values[j]
	values[j] = temp

def bubble_sort(values,N):
	for i in range(N):
		for j in range(N-i-1):
			if(values[j+1][2]>values[j][2]):
				swap(j,j+1,values)
			elif(values[j+1][2]==values[j][2]):
				if((values[j+1][0]+values[j+1][1])<(values[j][0]+values[j][1])):
					swap(j,j+1,values)

def merge_sort(values):
	if(len(values) > 1):
		mid  = len(values)//2
		l =  values[:mid]
		r = values[mid:]
		merge_sort(l)
		merge_sort(r)
		il= 0
		ir= 0 
		i = 0
		while(il < len(l) and ir <len(r)):
			if( l[il][2] > r[ir][2]):
				values[i] = l[il]
				il +=1
			elif(l[il][2] == r[ir][2]):
				if(l[il][0]+l[il][1]).lower() < (r[ir][0] +r[ir][1]).lower():
					values[i]  = l[il]
					il +=1
				else:
					values[i]  = r[ir]
					ir +=1
			else:
				values[i] = r[ir]
				ir +=1
			i+=1
		
		while( il < len(l)):
			values[i] = l[il]
			il += 1
			i+=1
			
		while( ir<len(r)):
			values[i] = r[ir]
			ir += 1
			i+=1

### 7.2 

**Complexity**

__SelectionSort__ : Given an array of size N, the selection sort algorithm, in the worst case scenario that the array is ordered in the opposite ordering we want, makes for every element of the array in position i, N-i-1 comparisons (i goes from 0 to N-1). Then swaps its position with the one of entry with the biggest mean in the section of the array that has not been ordered yet. 


*Number of comparisons* = $\sum_{i = 1}^{N}$ $\sum_{j=i+1}^{N} 1$ = $\sum_{i=1}^{N}(N-(i+1)+1)$ = $\sum_{i=1}^{N}(N-i)$ = $N(N-1)$ - $\frac{N(N-1)}{2}$ = $\frac{N(N-1)}{2}$ = $O$($N$^${2}$)

__BubbleSort__ : Given an array of size N, the bubble sort algorithm, in the worst case scenario that the array is ordered in the opposite ordering we want, the bubble sort makes $\frac{N^{2}}{2}$
comparisons and swaps. So has a complexity equal to $O$($N^{2}$)

__MergeSort__ : Given an array of size N,N>1, the merge sort algorithm makes several steps to sort the algorithm. It's a recursive algorithm:
At the first step (i=1) merges array of size 1, at the second step (i=2) merges array of size 2, at the i-th step merges array of size $2^{i-1}$.
To merge two arrays of same size N takes only one loop over the two array so it take $O$($N$).
The last step occurs when merges array of size  $\frac{N}{2}$, so the last step occurs when $2^{i-1}$ = $\frac{N}{2}$   ==> i-1 = $\log{_2}{\frac{N}{2}}$ ==> i is $O$($\log{N}$)
In conclusion the algorithm has to merge two arrays $\log{N}$ times, so has time complexity $O$(N $\log{N}$)

### 7.3

In [7]:
from sorting import *
import time

In [8]:
print('Insert how many rows you want to sort, insert 0 to read all the text rows.')
N = int(input())
start = time.time()
rows,N = readTxt(N)
end  =  time.time()	
print("Time to read N = "+str(N)+" rows: "+str(end-start)+" seconds")

Insert how many rows you want to sort, insert 0 to read all the text rows.
Time to read N = 50000 rows: 10.293111085891724 seconds


In [9]:
start = time.time()
unsorted_sel=  rows
sel_sort(unsorted_sel,N)
end  =  time.time()	
sel_time = end-start
print("Selection takes: "+str(sel_time)+" seconds to sort N = "+str(N)+" rows!")

Selection takes: 496.15750432014465 seconds to sort N = 50000 rows!


In [10]:
start = time.time()
unsorted_bubble=  rows
bubble_sort(unsorted_bubble,N)
end  =  time.time()	
bub_time = end-start
print("Bubble Sort takes: "+str(bub_time)+" seconds to sort N = "+str(N)+" rows!")

Bubble Sort takes: 1293.1193449497223 seconds to sort N = 50000 rows!


In [11]:
start = time.time()
unsorted_merge = rows
merge_sort(unsorted_merge)
end  =  time.time()	
merge_time = end-start
print("Merge Sort takes: "+str(merge_time)+" seconds to sort N = "+str(N)+" rows!")

Merge Sort takes: 0.5184693336486816 seconds to sort N = 50000 rows!


In [12]:
def write_Txt(sorted_values):
    with open('RankingLists.txt','w') as output:
        for elem in sorted_values:
            output.write(elem[0]+' '+elem[1]+' '+str(elem[2])+'\n')


In [13]:
write_Txt(unsorted_merge)

### 7.4

The most time efficient algorithm between the 3 we implemented is the __mergeSort__ algorithm. This algorithm has a time complexity much smaller than the complexity of __bubbleSort__ and __selectionSort__. This statement is verified just by seeing that to complete the query on the ApplicantsInfo.txt the __mergeSort__ completes the task in less than a second ( 0.52 seconds), while the __selectionSort__ needs about  8 minutes and 16.16 seconds (496.16 seconds) and the  __bubbleSort__ needs 21 minutes and 33.12 seconds (1293.12 seconds)!

The worst time efficient algorithm is the __bubbleSort__, even though it has the same time complexity as the __selectionSort__.
However the __bubbleSort__ needs almost the double of the time needed by the __selectionSort__ to complete the task. This occurs because the __bubbleSort__ makes more swap operation, roughly the double, of entries than the __selectionSort__.

### 7.5

**MapReduce Implementations**

In [14]:
from collections import Counter

In [15]:
def mapReduce(values):
    counter  = Counter()
    for elem in values:
        v= elem[2]
        k = elem[0]+' '+elem[1]
        counter[k] = v
    
    return counter
    


In [16]:
start  =time.time()
unsorted_map = rows
mapReduce(unsorted_map)
end = time.time()
print("MapReduce takes "+str(end-start)+" seconds to sort "+str(N)+" rows")

MapReduce takes 0.04696989059448242 seconds to sort 50000 rows


My __MapReduce__ algorithm implementation, which uses the counter data structure from the collections module, is even faster than the __mergeSort__ algorithm. 

__MapReduce__ needs only 0.047 seconds to sort 50000 rows, while __mergeSort__ algorithm needs 0.52 seconds, so __mapReduce__ is one order of magnitude faster than the __mergeSort__ algorithm.